In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/2020 WINTER PROJ/input/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [ ]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
def remove_space(text):
    text = text.strip()
    text = text.split()
    return " ".join(text)
train['text'] = train['text'].apply(lambda x:remove_space(str(x)))

In [ ]:
train['text'] = train['text'].apply(lambda x:str(x).lower())

0                      i`d have responded, if i were going
1            sooo sad i will miss you here in san diego!!!
2                                my boss is bullying me...
3                           what interview! leave me alone
4        sons of ****, why couldn`t they put them on th...
                               ...                        
27476    wish we could come see u on denver husband los...
27477    i`ve wondered about rake to. the client has ma...
27478    yay good for both of you. enjoy the break - yo...
27479                            but it was worth it ****.
27480    all this flirting going on - the atg smiles. y...
Name: text, Length: 27481, dtype: object

In [ ]:
# Download WordNet data
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
import re
from nltk.corpus import wordnet

# Correct the words with repeated characters
# ref: https://www.youtube.com/watch?v=r37OYsdH6Z8
class RepeatReplacer:
  def __init__(self):
    self.regex = re.compile(r'(\w*)(\w)\2(\w*)')
    self.repl = r'\1\2\3'

  def replace(self, word):
    if wordnet.synsets(word):
      return word
    loop_res = self.regex.sub(self.repl, word)
    if (word == loop_res):
      return loop_res
    else:
      return self.replace(loop_res)

In [13]:
# Testing RepeatReplacer
replacer = RepeatReplacer()
replace_result = replacer.replace('Sweeeeeeeet')

print(replace_result)

Sweet


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
train.text = train.text.apply(lambda x:' '.join([lmtzr.lemmatize(word, 'v') for word in str(x).split()]))

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
train['words'] = train['text'].apply(lambda x:str(x).split())

from nltk.corpus import stopwords
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train.words = train.words.apply(lambda x:remove_stopword(x))

In [ ]:
contraction = {
    "'cause": 'because',
    ',cause': 'because',
    ';cause': 'because',
    "ain't": 'am not',
    'ain,t': 'am not',
    'ain;t': 'am not',
    'ain`t': 'am not',
    'aint': 'am not',
    'arent': 'are not',
    "aren't": 'are not',
    'are;t': 'are not',
    
}